# Importação das bibliotecas necessárias

In [1]:
import sys
import os
sys.path.append('/home/jovyan/work')

from utils.AlinharETL import AlinharETL

# Crie uma instância da classe AlinharETL

In [2]:
# Crie uma instância da classe
bucket = "bronze"
datamart = "FTPMicrodadosNovoCaged"
extrator_bronze = AlinharETL(bucket,datamart)

2024-09-25 19:03:53,205 - INFO - Iniciando Sessão Spark.


# Leitura da tabela

In [3]:
df = extrator_bronze.criar_view_temporaria('bronze/NovoCaged/MicrodadosNovoCagedSubClasse','compliance_subclasse')

2024-09-25 19:03:57,257 - INFO - Aguarde... Criando View (bronze/NovoCaged/MicrodadosNovoCagedSubClasse)
2024-09-25 19:04:06,125 - INFO - View criada com sucesso


# Tratamentos na tabela 

In [4]:
sql_query = """
    SELECT 
        CAST(`Código` AS INT) AS codigo, 
        `Descrição` AS descricao 
    FROM compliance_subclasse
"""

In [5]:
df = extrator_bronze.carregar_dados_delta(sql_query)

2024-09-25 19:04:06,138 - INFO - Aguarde... Executando Query (Delta)
2024-09-25 19:04:06,140 - INFO - 
    SELECT 
        CAST(`Código` AS INT) AS codigo, 
        `Descrição` AS descricao 
    FROM compliance_subclasse

2024-09-25 19:04:06,234 - INFO - Query (Delta) executada com sucesso


# Gravação no datalake

In [6]:
extrator_bronze.caminho_tabela_delta = 's3a://silver/NovoCaged/MicrodadosNovoCagedSubClasse'

In [7]:
extrator_bronze.salvar_delta(df, 'overwrite')

2024-09-25 19:04:06,250 - INFO - Aguarde... Persistindo dados (overwrite)
2024-09-25 19:04:16,407 - INFO - Dados persistidos com sucesso
2024-09-25 19:04:16,409 - INFO - s3a://silver/NovoCaged/MicrodadosNovoCagedSubClasse
2024-09-25 19:04:16,409 - INFO - Aguarde... Realizando optimize
2024-09-25 19:04:19,057 - INFO - Optimize executado com sucesso.
2024-09-25 19:04:19,058 - INFO - Aguarde... Realizando vacuum
2024-09-25 19:04:47,032 - INFO - Vacuum executado com sucesso.


# Encerra sessão spark

In [8]:
extrator_bronze.parar_sessao()

2024-09-25 19:04:47,562 - INFO - Sessão Spark finalizada.
